In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)

In [2]:
dval = dmyplant2.Validation.load_def_csv("input2.csv")
dval;

In [3]:
i = 2
vale = dval.iloc[i][['serialNumber','Validation Engine','val start']]

In [4]:
e=dmyplant2.Engine.from_sn(mp,vale['serialNumber'], valstart=vale['val start'])
#e=dmyplant2.Engine.from_sn(mp,1486144) # Forsa Hartmoor M1
#e=dmyplant2.Engine.from_sn(mp,1486152) # Forsa Hartmoor M2
#e=dmyplant2.Engine.from_sn(mp,1486176) # Forsa Hartmoor M3
#e=dmyplant2.Engine.from_sn(mp,1486174) # Forsa Hartmoor M4
#e=dmyplant2.Engine.from_sn(mp,1486197) # Forsa Hartmoor M5
#e=dmyplant2.Engine.from_sn(mp,1486198) # Forsa Hartmoor M6
#e=dmyplant2.Engine.from_sn(mp,1486236) # Forsa Hartmoor M7
#e=dmyplant2.Engine.from_sn(mp,1486225) # Forsa Hartmoor M8
#e=dmyplant2.Engine.from_sn(mp,1486251) # Forsa Hartmoor M9
#e=dmyplant2.Engine.from_sn(mp,1486255) # Forsa Hartmoor M10
#e=dmyplant2.Engine.from_sn(mp,1486272) # Forsa Hartmoor M11
print(f"loadramp: {e['rP_Ramp_Set']} %/sec => {100.0 / e['rP_Ramp_Set']:4.1f} sec.")
pd.DataFrame.from_dict(e.dash, orient='index').T

loadramp: 0.58 %/sec => 172.4 sec.


,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,ROCHE PENZBERG KWKK,M01,U233,624,H02,24,4489.3,24.0,1184199,108532,39563.0,2020-04-27,0,39563,0.017


In [5]:
from dfsm import msgFSM
fsm = msgFSM(e, skip_days=7)
fsm.run() # run Finite State Machine

Scan Messages: 100%|██████████████████████████████████████████████████| 120891/120891 [00:09<00:00, 13001.56 messages/s]


In [6]:
rdf = pd.DataFrame(fsm._starts)
rda = rdf
#rda = rdf[((rdf['mode'] == 'AUTO') & (rdf['success'] == True))]
#rda = rdf[((rdf['mode'] == 'AUTO') & (rdf['success'] == True) & (rdf['synchronize'] < 300.0) & (rdf['start-preparation'] < 300.0))]
#rda = rdf[((rdf['success'] == True) & (rdf['start-preparation'] < 1000.0))]
rda = rdf[((rdf['success'] == True))]
rda = rda[['success','mode','cumstarttime','start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']]
rda.round(2)

,success,mode,cumstarttime,start-preparation,starter,hochlauf,idle,synchronize,load-ramp,target-operation
20,True,MANUAL,506.31,256.12,7.07,16.48,50.58,3.66,172.41,0 days 00:44:06
21,True,AUTO,362.71,126.08,4.65,18.24,9.73,31.60,172.41,0 days 00:27:02
22,True,AUTO,394.65,162.55,4.64,18.79,9.72,26.53,172.41,0 days 00:08:41
23,True,MANUAL,438.21,205.20,5.07,18.09,9.00,28.43,172.41,0 days 00:58:32
25,True,MANUAL,491.10,253.70,5.15,19.29,9.61,30.93,172.41,0 days 00:48:48
...,...,...,...,...,...,...,...,...,...,...
1051,True,AUTO,372.95,141.57,5.34,20.04,6.76,26.83,172.41,1 days 16:35:21
1052,True,MANUAL,385.77,155.92,5.82,21.07,7.25,23.30,172.41,0 days 00:12:56
1053,True,MANUAL,366.18,135.02,6.14,19.95,7.07,25.59,172.41,0 days 00:07:06
1054,True,MANUAL,1083.59,852.04,6.24,20.25,7.43,25.22,172.41,6 days 16:27:56


In [7]:
rdb = rda[rdf['cumstarttime'] > 300.0]
rdb.round(2)

,success,mode,cumstarttime,start-preparation,starter,hochlauf,idle,synchronize,load-ramp,target-operation
20,True,MANUAL,506.31,256.12,7.07,16.48,50.58,3.66,172.41,0 days 00:44:06
21,True,AUTO,362.71,126.08,4.65,18.24,9.73,31.60,172.41,0 days 00:27:02
22,True,AUTO,394.65,162.55,4.64,18.79,9.72,26.53,172.41,0 days 00:08:41
23,True,MANUAL,438.21,205.20,5.07,18.09,9.00,28.43,172.41,0 days 00:58:32
25,True,MANUAL,491.10,253.70,5.15,19.29,9.61,30.93,172.41,0 days 00:48:48
...,...,...,...,...,...,...,...,...,...,...
1051,True,AUTO,372.95,141.57,5.34,20.04,6.76,26.83,172.41,1 days 16:35:21
1052,True,MANUAL,385.77,155.92,5.82,21.07,7.25,23.30,172.41,0 days 00:12:56
1053,True,MANUAL,366.18,135.02,6.14,19.95,7.07,25.59,172.41,0 days 00:07:06
1054,True,MANUAL,1083.59,852.04,6.24,20.25,7.43,25.22,172.41,6 days 16:27:56


In [8]:
rdb.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
cumstarttime,728.0,399.45,80.73,340.88,357.91,374.67,394.66,1088.87
start-preparation,728.0,158.2,50.92,120.62,126.65,147.35,152.7,852.04
starter,728.0,5.74,1.15,4.25,4.95,5.24,6.87,10.8
hochlauf,728.0,19.91,3.8,10.6,18.1,19.52,21.16,93.41
idle,728.0,23.67,53.81,6.34,7.33,9.21,10.53,527.14
synchronize,728.0,19.51,8.92,0.5,14.49,19.99,25.1,113.36
load-ramp,728.0,172.41,0.0,172.41,172.41,172.41,172.41,172.41
target-operation,727,2 days 05:21:10.557083906,4 days 09:40:51.308662081,0 days 00:00:02,0 days 00:34:09,0 days 16:10:34,2 days 10:43:03.500000,53 days 07:23:42


In [9]:
rdc = rda[rdf['load-ramp'] < 120.0]
rdc.round(2)

,success,mode,cumstarttime,start-preparation,starter,hochlauf,idle,synchronize,load-ramp,target-operation


In [10]:
rdc.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
cumstarttime,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT
start-preparation,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT
starter,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT
hochlauf,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT
idle,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT
synchronize,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT
load-ramp,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT
target-operation,0,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [11]:
print("""
****************
** Überblick: **
****************
""")
print(f"{'Motor':9}: {e}")
print(f"{'Meldung':9}- '{'9047 target load reached'}' {'gefunden.' if any(fsm._messages['name'] == '9047') else 'wird berechnet.'}")
print(f"{'Zeitraum':9}: {fsm.first_message:%d.%m.%Y} bis {fsm.last_message:%d.%m.%Y}")
print(f"{'Dauer':9}: {fsm.period.days} Tage")
print()
print(f"{'GESAMT':9}: {rdf.shape[0]:5} Starts, {rdf[rdf.success].shape[0]:5} erfolgreich => {(rdf[rdf.success].shape[0] / rdf.shape[0]) * 100.0:3.1f} % der Starts erfolgreich.")
for mode in ['MANUAL', 'AUTO']:
    print(f"{mode:9}: {rdf[(rdf['mode'] == mode)].shape[0]:5} Starts, {rdf[((rdf.success) & (rdf['mode'] == mode))].shape[0]:5} erfolgreich => {(rdf[((rdf.success) & (rdf['mode'] == mode))].shape[0] / rdf[(rdf['mode'] == mode)].shape[0]) * 100.0:3.1f} % der Starts erfolgreich.")


****************
** Überblick: **
****************

Motor    : 1184199 M01 ROCHE PENZBERG KWKK    
Meldung  - '9047 target load reached' berechnet.
Zeitraum : 01.07.2016 bis 16.02.2022
Dauer    : 2063 Tage

GESAMT   :  1056 Starts,   728 erfolgreich => 68.9 % der Starts erfolgreich.
MANUAL   :   197 Starts,    69 erfolgreich => 35.0 % der Starts erfolgreich.
AUTO     :   746 Starts,   584 erfolgreich => 78.3 % der Starts erfolgreich.


In [ ]:
rda.describe().round(2).T

In [ ]:
rda[['start-preparation','starter','hochlauf','idle','synchronize','cumstarttime','load-ramp','target-operation']].hist(figsize = (20,12), bins=50, layout=(3,3));
#rda.hist(figsize = (20,12), bins=100, layout=(4,2));

In [ ]:
rda.plot.box(subplots=True, grid = True, figsize=(24,10), layout=(1,7));

In [ ]:
rda.plot.box(subplots=True, grid = True, figsize=(24,10), layout=(1,7), sym='');

In [ ]:
nalarms = []
ct = 0
for i,c in rdf.iterrows():
    if len(c['alarms']) > 0 and not c['success']:
        ct += 1
        print(f"\nStartversuch: {i}, Success: {c['success']}")
        for a in c['alarms']:
            nalarms.append(a['msg'])
            print(f"{c['mode']:15} {a['state']:20} {pd.to_datetime(int(a['msg']['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {a['msg']['name']} {a['msg']['message']}")
print(f"""
***********************************
** {ct:3} nicht erfolgreiche Starts **
***********************************
""")

In [ ]:
al = pd.DataFrame(fsm._pareto(nalarms))
fig = plt.figure();
color = 'purple'

if not al.empty:
    al['msg'] = al['msg'] + ' (' + al['name'] + ')'
    al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title='Alarms in not successful Starts');
    plt.plot();

In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'red'

for phase in phases:
    al = fsm.alarms_pareto(phase)[:20]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'orange'

for phase in phases:
    al = fsm.warnings_pareto(phase)[:20]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
mfn = './data/' + str(e._sn) + '_messages.txt'
fsm.save_messages('./data/' + str(e._sn) + '_messages.txt')
print(mfn)

In [ ]:
any(fsm._messages['name'] == '9047')

In [ ]:
#fsm.store()